In [2]:
import pandas as pd
import nltk
import re

import glob
import io
import numpy as np

## Job Titles list

In [3]:
with open('CityofLA/Job Bulletins/311 DIRECTOR  9206 041814.txt', encoding = 'utf-8') as f:
    print(f.readlines())

['311 DIRECTOR\n', 'Class Code:       9206\n', 'Open Date:  04-18-14\n', 'ANNUAL SALARY \n', '\n', '$125,175 to $155,514 \n', '\n', 'DUTIES\n', '\n', "A 311 Director is responsible for the successful operation and expansion of the 311 Call Center in the Information Technology Agency (ITA) which answers call from constituents regarding Citywide services provided by City departments; works to ensure the efficient and effective resolution of any issues that may arise; plans, directs, hires, coaches, and coordinates a large staff of professional, technical and clerical employees engaged in the implementation, administration, and operations of the City's 311 Call Center; applies sound supervisor principles and techniques in building and maintaining and effective work force; fulfills equal opportunity responsibilities; and does related work.\n", '\n', 'REQUIREMENTS\n', '\n', '1. One year of full-time paid experience as a Senior Management Analyst with the City of Los Angeles or in a class wh

In [4]:
with open('CityofLA/Additional data/kaggle_data_dictionary.csv', encoding = 'utf-8') as f:
    print(f.readlines())

['Field Name,Annotation Letter,Description,Data Type,Allowable Values,Accepts Null Values?,Additional Notes\n', 'FILE_NAME,,The file name of the job bulletin from which each record came,String,,No,\n', 'JOB_CLASS_TITLE,A,"The title of the particular class (e.g., Systems Analyst, Carpenter)",String,Any job title matching in in supplied job bulletins,No,\n', 'JOB_CLASS_NO,B,The 4-digit code that corresponds to the job class.,String,Any four character numeric string matching one in supplied job bulletins,No,"Some classes have three non-zero digits — e.g., 0162. These will take the form 0###"\n', 'REQUIREMENT_SET_ID,D1,"A main requirement that has to be filled. These are identified with numbers in the job bulletin. Relationships between requirement sets are specified by an ""AND|OR"" conjunction (data element E1 on the data annotation chart)",Integer,Any positive integer,No,\n', 'REQUIREMENT_SUBSET_ID,D2,"A secondary requirement that has to be filled, nested under a REQUIREMENT_SET_ID. The

In [5]:
job_titles = pd.read_csv('CityofLA/Additional data/job_titles.csv')

## Data Dictionary

In [5]:
kaggle_data_dictionary = pd.read_csv('CityofLA/Additional data/kaggle_data_dictionary.csv')

In [6]:
kaggle_data_dictionary

,Field Name,Annotation Letter,Description,Data Type,Allowable Values,Accepts Null Values?,Additional Notes
0,FILE_NAME,NaN,The file name of the job bulletin from which e...,String,NaN,No,NaN
1,JOB_CLASS_TITLE,A,"The title of the particular class (e.g., Syste...",String,Any job title matching in in supplied job bull...,No,NaN
2,JOB_CLASS_NO,B,The 4-digit code that corresponds to the job c...,String,Any four character numeric string matching one...,No,Some classes have three non-zero digits — e.g....
3,REQUIREMENT_SET_ID,D1,A main requirement that has to be filled. Thes...,Integer,Any positive integer,No,NaN
4,REQUIREMENT_SUBSET_ID,D2,"A secondary requirement that has to be filled,...",String,Any letter or combination of letters.,No,"A-Z represents 1-26. In the case of 27th, 28th..."
5,JOB_DUTIES,C,A summary of what someone does in the particul...,String,NaN,No,NaN
6,EDUCATION_YEARS,F,Years required in a particular education program,Float,Any positive float representing years or a fra...,Yes,Any job description that mentions months shoul...
7,SCHOOL_TYPE,G,School Type: School type required (e.g. colleg...,String,"COLLEGE OR UNIVERSITY, HIGH SCHOOL, APPRENTICE...",Yes,Allowable value list is most likely incomplete...
8,EDUCATION_MAJOR,H,Major or concentration required,String,NaN,Yes,A list of many sufficient majors should appear...
9,EXPERIENCE_LENGTH,I,Years required in a particular job class or ex...,Float,NaN,Yes,Some classes may specify months rather than ye...


In [7]:
kaggle_data_dictionary.iloc[3]['Description']

'A main requirement that has to be filled. These are identified with numbers in the job bulletin. Relationships between requirement sets are specified by an "AND|OR" conjunction (data element E1 on the data annotation chart)'

## Sample 

In [9]:
sample = pd.read_csv('CityofLA/Additional data/sample job class export template.csv')

In [10]:
sample.head()

,FILE_NAME,JOB_CLASS_TITLE,JOB_CLASS_NO,REQUIREMENT_SET_ID,REQUIREMENT_SUBSET_ID,JOB_DUTIES,EDUCATION_YEARS,SCHOOL_TYPE,EDUCATION_MAJOR,EXPERIENCE_LENGTH,...,COURSE_LENGTH,COURSE_SUBJECT,MISC_COURSE_DETAILS,DRIVERS_LICENSE_REQ,DRIV_LIC_TYPE,ADDTL_LIC,EXAM_TYPE,ENTRY_SALARY_GEN,ENTRY_SALARY_DWP,OPEN_DATE
0,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,1,A,"A Systems Analyst analyzes procedures, methods...",4.0,COLLEGE OR UNIVERSITY,COMPUTER SCIENCE|INFORMATION SYSTEMS|GEOGRAPHI...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17
1,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,2,A,"A Systems Analyst analyzes procedures, methods...",4.0,COLLEGE OR UNIVERSITY,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17
2,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,2,B,"A Systems Analyst analyzes procedures, methods...",4.0,COLLEGE OR UNIVERSITY,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17
3,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,2,C,"A Systems Analyst analyzes procedures, methods...",4.0,COLLEGE OR UNIVERSITY,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17
4,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,3,A,"A Systems Analyst analyzes procedures, methods...",NaN,NaN,NaN,2.0,...,"3M, 4Q",INFORMATION SYSTEMS|SYSTEMS ANALYSIS|closely r...,At least three of the courses must be from the...,M,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17


In [31]:
text = ''
with open('CityofLA/Job Bulletins/311 DIRECTOR  9206 041814.txt', encoding = 'utf-8') as f:
    text = f.read()

In [12]:
job_title = text.splitlines()[0].strip()

In [55]:
code = r'Class Code:\s*(\d*)'
open_d = r'Open Date:\s*(\d\d-\d\d-\d\d)'
sal = r'(\$(\d+,\d+))((\s(to|and|-)\s)(\$\d+,\d+))?' # Taken from kaggle kernel
sal_dwp = r'Power\sis\s((\$(\d+,\d+))((\s(to|and|-)\s)(\$\d+,\d+))?)'
dut = r'DUTIES\W+(.*\n)'
req = r'REQUIREMENT(S)?(/MINIMUM\sQUALIFICATION)?\W+(.*\n)'
end_d = r'(MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY)\W+(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)(.*\d)'
#open_d = r'Open Date: (.*)\n'

In [56]:
def doextraction(glob_text):
    col = ['FILE_NAME', 'JOB_CLASS_TITLE', 'JOB_CLASS_NO', 'SALARY_START', 'SALARY_END',
                   'ENTRY_SALARY_DWP','DUTIES', 'REQUIREMENTS', 'APPLICATION_DEADLINE', 'OPEN_DATE']
    df = pd.DataFrame(columns = col) 
    """Get all the files from the given glob and pass them to the extractor."""
    for thefile in glob.glob(glob_text)[:200]:
        with io.open(thefile, 'r', encoding='utf-8') as fyl:
            text = fyl.read()
            df = get_features(text, thefile, df)
            
    return df

In [61]:
def get_features(text, filename, df):
    filename = filename.replace('CityofLA/Job Bulletins\\', '')
    headings = []
    for line in text.split('\n'):
        if line.isupper():
            headings.append(line.strip())
    job_title = text.strip().splitlines()[0].strip()
    class_code = re.findall(code, text)[0]
    try:
        open_date = re.findall(open_d, text)[0]
    except:
        open_date = np.NaN
    salary_range = re.search(sal, text)
    
    try:
        salary_start = salary_range.group(1)
    except:
        salary_start = np.NaN
        
    try:
        salary_end = salary_range.group(6)
    except:
        salary_end = np.NaN
    salary_dwp = re.search(sal_dwp, text).group(1) if re.search(sal_dwp, text) is not None else np.NaN
    x = headings.index([elm for elm in headings if elm.startswith('REQUIREMENT')][0])
    m = re.search(headings[x], text).end()
    n = re.search(headings[x+1], text).start()
    requirement = text[m:n].strip()
    try:
        x = headings.index([elm for elm in headings if elm.startswith('DUT')][0])
        m = re.search(headings[x], text).end()
        n = re.search(headings[x+1], text).start()
        duties = text[m:n].strip()
    except:
        duties = np.NaN
    try:
        x = re.findall(end_d, text)[0]
        deadline = ''.join(x).strip()
    except:
        deadline = np.NaN
        
    df = df.append({'FILE_NAME': filename, 'JOB_CLASS_TITLE': job_title, 'JOB_CLASS_NO': class_code, 'SALARY_START': salary_start,\
                    'SALARY_END': salary_end, 'ENTRY_SALARY_DWP': salary_dwp,'DUTIES': duties, 'REQUIREMENTS': requirement, 'APPLICATION_DEADLINE': deadline,\
                    'OPEN_DATE': open_date}, ignore_index = True)
    
    return df

In [62]:
data_df = doextraction('CityofLA/Job Bulletins/*.txt')

In [63]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
data_df

,FILE_NAME,JOB_CLASS_TITLE,JOB_CLASS_NO,SALARY_START,SALARY_END,ENTRY_SALARY_DWP,DUTIES,REQUIREMENTS,APPLICATION_DEADLINE,OPEN_DATE
0,311 DIRECTOR 9206 041814.txt,311 DIRECTOR,9206,"$125,175","$155,514",NaN,A 311 Director is responsible for the successf...,1. One year of full-time paid experience as a ...,"THURSDAYMAY 1, 2014",04-18-14
1,ACCOUNTANT 1513 062218.txt,ACCOUNTANT,1513,"$49,903","$72,996",NaN,An Accountant does professional accounting wor...,Graduation from an accredited four-year colleg...,"SATURDAYAUGUST 25, 2018",06-22-18
2,ACCOUNTING CLERK 1223 071318.txt,ACCOUNTING CLERK,1223,"$49,005","$71,618",NaN,An Accounting Clerk performs difficult and res...,Two years of full-time paid office clerical ex...,NaN,07-13-18
3,ACCOUNTING RECORDS SUPERVISOR 1119 072718.txt,ACCOUNTING RECORDS SUPERVISOR,1119,"$55,332","$80,930",NaN,"An Accounting Records Supervisor assigns, revi...",Two years of full-time paid experience as an A...,"THURSDAYAUGUST 9, 2018",07-27-18
4,ADMINISTRATIVE ANALYST 1590 060118.txt,ADMINISTRATIVE ANALYST,1590,"$60,489","$88,468",NaN,An Administrative Analyst performs professiona...,1. One year of full-time paid professional exp...,"THURSDAYJUNE 14, 2018",06-01-18
5,ADMINISTRATIVE CLERK 1358 033018 (2).txt,ADMINISTRATIVE CLERK,1358,"$37,584","$54,935","$43,785",An Administrative Clerk performs general offic...,Six months of full-time paid experience in cle...,"THURSDAYAPRIL 12, 2018",03-30-18
6,ADMINISTRATIVE HEARING EXAMINER 9135 100915.txt,ADMINISTRATIVE HEARING EXAMINER,9135,"$54,726","$80,012",NaN,An Administrative Hearing Examiner conducts qu...,1. Two years of full-time paid experience with...,"THURSDAYOCTOBER 22, 2015",10-09-15
7,ADVANCE PRACTICE PROVIDER CORRECTIONAL CARE 23...,ADVANCE PRACTICE PROVIDER CORRECTIONAL CARE,2325,"$78,487","$97,530",NaN,An Advance Practice Provider Correctional Care...,1. Must be licensed as a: \na. Registered Nurs...,NaN,02-08-08
8,AIR CONDITIONING MECHANIC 3774 041417.txt,AIR CONDITIONING MECHANIC,3774,"$90,118",None,"$99,054","An Air Conditioning Mechanic installs, modifie...",1. Completion of a recognized apprenticeship a...,"THURSDAYAPRIL 27, 2017",04-14-17
9,AIR CONDITIONING MECHANIC SUPERVISOR 3781 1116...,AIR CONDITIONING MECHANIC SUPERVISOR,3781,"$103,841","$108,482","$127,388 to $134,488",An Air Conditioning Mechanic Supervisor assign...,Four years of full-time paid experience as an ...,"THURSDAYNOVEMBER 29, 2018",11-16-18


In [37]:
exp_len = r'(one|two|three|four|five|six|seven|eight|nine)\s(?=(year|years|month|months)\sof\s(?=(full-time|part-time)))'
data_df['EXPERIENCE_LENGTH'] = data_df.REQUIREMENTS.apply(lambda x: re.search(exp_len,x, re.IGNORECASE).group(1)+ ' ' + re.search(exp_len,x, re.IGNORECASE).group(2) \
                                                          if re.search(exp_len,x, re.IGNORECASE) is not None else np.NaN)
data_df['Full-time/part-time'] = data_df.REQUIREMENTS.apply(lambda x: re.search(exp_len,x,re.I).group(3) if re.search(exp_len,x,re.I) is not None else np.NaN)

In [65]:
data_df

,FILE_NAME,JOB_CLASS_TITLE,JOB_CLASS_NO,SALARY_START,SALARY_END,ENTRY_SALARY_DWP,DUTIES,REQUIREMENTS,APPLICATION_DEADLINE,OPEN_DATE,SCHOOL_TYPE,EDUCATION_YEARS,EDUCATION_MAJOR
0,311 DIRECTOR 9206 041814.txt,311 DIRECTOR,9206,"$125,175","$155,514",NaN,A 311 Director is responsible for the successf...,1. One year of full-time paid experience as a ...,"THURSDAYMAY 1, 2014",04-18-14,college,NaN,a call
1,ACCOUNTANT 1513 062218.txt,ACCOUNTANT,1513,"$49,903","$72,996",NaN,An Accountant does professional accounting wor...,Graduation from an accredited four-year colleg...,"SATURDAYAUGUST 25, 2018",06-22-18,college,four-year,NaN
2,ACCOUNTING CLERK 1223 071318.txt,ACCOUNTING CLERK,1223,"$49,005","$71,618",NaN,An Accounting Clerk performs difficult and res...,Two years of full-time paid office clerical ex...,NaN,07-13-18,college,NaN,NaN
3,ACCOUNTING RECORDS SUPERVISOR 1119 072718.txt,ACCOUNTING RECORDS SUPERVISOR,1119,"$55,332","$80,930",NaN,"An Accounting Records Supervisor assigns, revi...",Two years of full-time paid experience as an A...,"THURSDAYAUGUST 9, 2018",07-27-18,NaN,NaN,NaN
4,ADMINISTRATIVE ANALYST 1590 060118.txt,ADMINISTRATIVE ANALYST,1590,"$60,489","$88,468",NaN,An Administrative Analyst performs professiona...,1. One year of full-time paid professional exp...,"THURSDAYJUNE 14, 2018",06-01-18,college,four-year,Management Aide
5,ADMINISTRATIVE CLERK 1358 033018 (2).txt,ADMINISTRATIVE CLERK,1358,"$37,584","$54,935","$43,785",An Administrative Clerk performs general offic...,Six months of full-time paid experience in cle...,"THURSDAYAPRIL 12, 2018",03-30-18,NaN,NaN,NaN
6,ADMINISTRATIVE HEARING EXAMINER 9135 100915.txt,ADMINISTRATIVE HEARING EXAMINER,9135,"$54,726","$80,012",NaN,An Administrative Hearing Examiner conducts qu...,1. Two years of full-time paid experience with...,"THURSDAYOCTOBER 22, 2015",10-09-15,NaN,NaN,NaN
7,ADVANCE PRACTICE PROVIDER CORRECTIONAL CARE 23...,ADVANCE PRACTICE PROVIDER CORRECTIONAL CARE,2325,"$78,487","$97,530",NaN,An Advance Practice Provider Correctional Care...,1. Must be licensed as a: \na. Registered Nurs...,NaN,02-08-08,NaN,NaN,NaN
8,AIR CONDITIONING MECHANIC 3774 041417.txt,AIR CONDITIONING MECHANIC,3774,"$90,118",None,"$99,054","An Air Conditioning Mechanic installs, modifie...",1. Completion of a recognized apprenticeship a...,"THURSDAYAPRIL 27, 2017",04-14-17,apprentice,NaN,refrigeration and
9,AIR CONDITIONING MECHANIC SUPERVISOR 3781 1116...,AIR CONDITIONING MECHANIC SUPERVISOR,3781,"$103,841","$108,482","$127,388 to $134,488",An Air Conditioning Mechanic Supervisor assign...,Four years of full-time paid experience as an ...,"THURSDAYNOVEMBER 29, 2018",11-16-18,NaN,NaN,NaN


In [44]:
data_df.EDUCATION_MAJOR.isnull().sum()

195

In [68]:
data_df[data_df.EDUCATION_MAJOR.notnull()]

,FILE_NAME,JOB_CLASS_TITLE,JOB_CLASS_NO,SALARY_START,SALARY_END,ENTRY_SALARY_DWP,DUTIES,REQUIREMENTS,APPLICATION_DEADLINE,OPEN_DATE,SCHOOL_TYPE,EDUCATION_YEARS,EDUCATION_MAJOR
8,AIR CONDITIONING MECHANIC 3774 041417.txt,AIR CONDITIONING MECHANIC,3774,"$90,118",None,"$99,054","An Air Conditioning Mechanic installs, modifie...",1. Completion of a recognized apprenticeship a...,"THURSDAYAPRIL 27, 2017",04-14-17,apprentice,NaN,refrigeration and air conditioning fitter
15,AIRPORT LABOR RELATIONS ADVOCATE 9210 020119.txt,AIRPORT LABOR RELATIONS ADVOCATE,9210,"$93,918","$137,327",NaN,The Airport Labor Relations Advocate advises s...,1. Graduation from an accredited college or un...,"THURSDAYFEBRUARY 14, 2019",02-01-19,college,NaN,private and public sector negotiating and admi...
24,AIRPORTS PUBLIC AND COMMUNITY RELATIONS DIRECT...,AIRPORTS PUBLIC AND COMMUNITY RELATIONS DIRECTOR,1788,"$86,777","$123,463",NaN,An Airports Public and Community Relations Dir...,1.\t\tTwo years of full-time paid professional...,"THURSDAYDECEMBER 21, 2017",12-08-17,college,four-year,supervising a public relations staff responsib...
34,AQUARIST 2400 050214.txt,AQUARIST,2400,"$55,624","$69,133",NaN,An Aquarist procures and maintains a variety o...,1. Graduation from a recognized four-year coll...,"THURSDAYMAY 15, 2014",05-02-14,college,four-year,a biological science
45,ART CURATOR 2448 071516 REV 072816.txt,ART CURATOR,2448,"$50,342","$73,581",NaN,An Art Curator performs professional curatoria...,1. A master's degree from an accredited four-y...,"THURSDAYAUGUST 4, 2016",07-15-16,college,four-year,"visual arts such as Art, Art History, Museum S..."
47,ARTS ASSOCIATE 2454 072117 REV 072817.txt,ARTS ASSOCIATE,2454,"$48,797","$71,326",NaN,An Arts Associate performs a broad range of en...,1. A bachelor's degree from an accredited four...,"THURSDAYAUGUST 3, 2017",07-21-17,college,four-year,"Arts (design, literary, media/communication, p..."
48,ARTS MANAGER 2455 092818.txt,ARTS MANAGER,2455,"$59,842","$87,508",NaN,"An Arts Manager assigns, reviews and evaluates...",1. Three years of full-time paid experience as...,"THURSDAYOCTOBER 11, 2018",09-28-18,college,four-year,"Art History, Art Administration, Architecture,..."
78,BLACKSMITH 3733 082517 REV 090717.txt,BLACKSMITH,3733,"$83,352",None,"$76,608 to $95,191","A blacksmith forges, shapes, forms, bends, cut...",1. Completion of an apprenticeship as a blacks...,"THURSDAYSEPTEMBER 14, 2017",08-25-17,apprentice,NaN,blacksmith
93,CABLE TELEVISION PRODUCTION MANAGER 1801 12141...,CABLE TELEVISION PRODUCTION MANAGER,1801,"$78,279","$114,443",NaN,A Cable Television Production Manager supervis...,1. Graduation from an accredited four-year col...,"THURSDAYDECEMBER 27, 2018",12-14-18,college,four-year,"Television/Film Production, Journalism, Commun..."
94,CARPENTER 3344 011218.txt,CARPENTER,3344,"$86,312",None,"$93,542",A Carpenter does skilled rough and finish carp...,1. Completion of an apprenticeship as a carpen...,"THURSDAYJANUARY 25, 2018",01-12-18,apprentice,NaN,carpenter or cabinetmaker


In [53]:
data_df.SALARY_END = data_df.SALARY_END.apply(lambda x: x if x is not None else '(flat-rated)')

In [27]:
sch_typ = r'(college|university|high school)'
m = re.search(sch_typ, "A Bachelor's degree from a recognized college or university", re.I)

In [20]:
m.group(1)

"Bachelor's"

In [67]:
sch_typ = r'(college|university|high school|apprentice)'
data_df['SCHOOL_TYPE'] = data_df.REQUIREMENTS.apply(lambda x: re.search(sch_typ, x, re.I).group(1)\
                                                    if re.search(sch_typ, x, re.I) is not None else np.NaN)
edu_y = r'(\b\w+(-year))(?=\D+(college|university))'
data_df['EDUCATION_YEARS'] = data_df.REQUIREMENTS.apply(lambda x: re.search(edu_y, x, re.I).group(1)\
                                                       if re.search(edu_y, x, re.I) is not None else np.NaN)

edu_major = r'(college|university|apprentice)\D+(in|as a)\s(\D+);'
data_df['EDUCATION_MAJOR'] = data_df.REQUIREMENTS.apply(lambda x: re.search(edu_major, x).group(3)\
                                                       if re.search(edu_major, x) is not None else np.NaN)

In [ ]:
edu = r'((bachelor|graduat)[A-Z]*)\D+((one|two|three|four|five|six|seven|eight|nine)-(year|years))(\D+(college|university)\D+((degree)\sin\s(\w+)))*'
((bachelor's|bachelor|graduat)[A-Z]*)*\D+(((one|two|three|four|five|six|seven|eight|nine)-(year|years))|(\d\s(semester|quarter)))*\D+(college|university)(\D+((degree)\sin\s(\w+)))*
  ((High School|bachelor|graduat)['a-z]*)\D+(?<=from)\D+(\s\w+(?=-year))(?:\D+)(college|university|high school)